# Overview

This file describes how to create a system as a graph.

This system is comprised of the following:

- a ZionEx host package
- a generic rack switch package
- a generic pod switch package
- a system that connects those packages in a clos fabric
- two ZionEx packages connected to 2 rack switches
- two rack switches connected to 2 pod switches

It also includes a sample use case that describes how host component paths in
the ZionEx host can be used to create collective communication optimizations.


# Define a ZionEx Package

This ZionEx class demonstrates how to take the ZionEx block diagram and capture the following:

- the package as a collection of components, links and adjacencies between them using `sys.proto`

![ZionEx Block Diagram](./meta-zion-ex.png)


# Create the ZionEx Device

- Create an instance of the ZionEx device
- Check the ZionEx device by creating a d3graph visualization


In [1]:
import sys

sys.path.append("..")
sys.path.append("../generated")
from zionex import ZionEx

zionex = ZionEx()
zionex.visualize().jupyter_display()

ModuleNotFoundError: No module named 'et_def_pb2'

# Define a generic Switch package

Create a class that models a generic switch which can be used in a fabric.


In [ ]:
"""Generic switch device

This can be used to create devices to be used in the formation of the
infrastructure fabric.
"""

from typing import Type
import infra_pb2
import builders


class Switch(builders.PackageBuilder):
    def __init__(self, name: str, down_links: int, up_links: int = 0):
        """
        Builds a generic switch device.

        Parameters
        ----------
        name: The name of the switch device
        down_links: The number of ports used for down link adjacencies
        up_links: The number of ports used for up link adjacencies
        """
        super().__init__()
        asic = infra_pb2.Component(
            name="asic",
            count=1,
            cpu=infra_pb2.Cpu(memory=infra_pb2.MemoryType.MEM_RAM),
        )
        self._port_down = infra_pb2.Component(
            name="port-down",
            count=down_links,
            nic=infra_pb2.Nic(ethernet=infra_pb2.Ethernet()),
        )
        mac = infra_pb2.Link(
            name="mac",
            type=infra_pb2.LinkType.LINK_CUSTOM,
        )
        self._device = infra_pb2.Device(
            name=name,
            components={
                asic.name: asic,
                self._port_down.name: self._port_down,
            },
            links={
                mac.name: mac,
            },
        )
        self._create_distributed_intra_package_adjacencies(
            asic,
            mac,
            self._port_down,
        )
        self._port_up = None
        if up_links > 0:
            self._port_up = infra_pb2.Component(
                name="port-up",
                count=up_links,
                nic=infra_pb2.Nic(ethernet=infra_pb2.Ethernet()),
            )
            self._device.components[self._port_up.name].CopyFrom(self._port_up)
            self._create_distributed_intra_package_adjacencies(
                asic,
                mac,
                self._port_up,
            )

    @property
    def port_down_component(self) -> Type[infra_pb2.Component]:
        return self._port_down

    @property
    def port_up_component(self) -> Type[infra_pb2.Component]:
        return self._port_up

# Create the Fabric Switch packages

- an instance of a rack switch with 8 down link ports and 8 up link ports
- an instance of a pod switch with 8 down link ports


In [ ]:
rack_switch = Switch("rack", down_links=8, up_links=8)
pod_switch = Switch("pod", down_links=8)

# Create the System

The system should consist of the following:

- 2 ZionEx packages
- 2 rack switch packages
- 2 pod switch packages
- adjacencies between ZionEx packages and rack switch packages
- adjacencies between rack switch packages and pod switch packages


In [ ]:
# use the helper ClosFabric class which will create a fabric and
# establish connections between the device instances
fabric = builders.ClosFabric(
    host_devices=infra_pb2.DeviceCount(count=2, device=zionex._device),
    rack_devices=infra_pb2.DeviceCount(count=2, device=rack_switch),
    pod_devices=infra_pb2.DeviceCount(count=2, device=pod_switch),
)

# visualize the entire system
fabric.visualize().jupyter_display()

# Communication Collective Optimization Use Case

Use Case:

> Communication between ranks MUST use internal paths if available
> and external paths if no internal path is available.

Example:

> 16 ranks over 2 ZionEx packages (1 rank per oam)

- for `rank0 to rank6` (ranks are in one ZionEx host)
  - determine the source component for rank0 and destination component for rank6
  - evaluate algorithms in order
  - the first algorithm `internal` yields a path `oam.0,oam-link,ep-oam-sw.0,oam-link,oam.7`
- for `rank1 to rank10` (ranks are in different ZionEx hosts)
  - determine the source component for rank1 and destination component for rank10
  - evaluate algorithms in order
  - the first algorithm `internal` yields no match
  - move to the next algorithm `external` which yields a path `oam.1,pcie4,ep-pcie-sw.0,pcie4,cc-pcie-sw.0,pcie3,nic.1`


# Create a Collective Optimization

The following creates a communication collective optimization for the `all to all` communication collective that consists of the following:

- an `internal` algorithm for communication between oam components in the same package
- an `external` algorithm for communication between oam components in different packages
- a `LIFO` schedule
